# Procesamiento

## Ejercicio 1

En el dataset de Aeropuertos (A), se debe generar una columna llamada **'elevation_name'** que contenga datos cualitativos indicando la elevación de cada aeropuerto. Esta columna se completará con las palabras: "bajo", "medio" o "alto", según la elevación de cada aeropuerto. Para definir estos grupos, se utilizarán los siguientes valores:

* bajo: aeropuertos con elevación menor o igual a 131 ft.
* medio: aeropuertos con elevación mayor que 131 ft y menor o igual a 903 ft.
* altos: aeropuertos con elevación mayor a 903 ft.

Además, se realizará otra modificación en este dataset, consistente en agregar una columna llamada **'prov_name'**, donde se incluirá el nombre de la provincia correspondiente a cada aeropuerto. Esta información se obtendrá consultando los nombres de las ciudades en el dataset (E).

In [1]:
import csv
from paths import AR_DATA, AR_AIRPORTS_DATA, AR_AIRPORTS_COPY_DATA
from preprocess_functions import classify_elevation, sanitize_text

# Abrir el archivo de entrada y el archivo de salida
infile_AR = open(AR_DATA, 'r', encoding='utf-8') 
infile_AR_AIRPORTS = open(AR_AIRPORTS_DATA, 'r', encoding='utf-8') 
reader_AR = csv.DictReader(infile_AR)
data_AR = list(reader_AR)
reader_AR_AIRPORTS = csv.DictReader(infile_AR_AIRPORTS)
with open(AR_AIRPORTS_COPY_DATA, 'w', newline='', encoding='utf-8') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=reader_AR_AIRPORTS.fieldnames + ['elevation_name', 'prov_name'])
    writer.writeheader()
            
    for row in reader_AR_AIRPORTS:
       
        elevation_name = classify_elevation(row)
        
        prov_name = ""
       
        for row_AR in data_AR:
            value_municipality = sanitize_text(row["municipality"])
            value_city = sanitize_text(row_AR["city"])
            if (value_municipality == value_city):
                prov_name = row_AR["admin_name"]
                break
            else:
                if row["region_name"] != "":
                    prov_name = row["region_name"].replace(' Province', '')
                else:
                    prov_name = "no_data"
        
        # Escribir la fila procesada en el archivo de salida
        row['elevation_name'] = elevation_name
        row['prov_name'] = prov_name
        writer.writerow(row)
infile_AR.close()
infile_AR_AIRPORTS.close()

## Ejercicio 2

En el dataset de Conectividad (B), se **realizará un reemplazo** en las celdas que contengan el carácter '-' con la palabra 'NO'. Para el mismo dataset se debe generar una nueva columna denominada **'posee_conectividad'**, la misma puede tomar uno de dos valores posibles: SÍ o NO.

El valor será NO si todos los campos ADSL, CABLEMODEM, DIALUP, FIBRAOPTICA, SATELITAL, WIRELESS, TELEFONIAFIJA, 3G y 4G poseen el valor --. Caso contrario el valor será SI

In [2]:
import csv
from paths import CONECTIVIDAD_DATA, CONECTIVIDAD_COPY_DATA 

with open(CONECTIVIDAD_DATA, 'r', newline='', encoding='utf-8') as infile, open(CONECTIVIDAD_COPY_DATA, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['posee_conectividad']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            # Reemplazar '--' con 'NO' en campos relevantes
            for field in ['ADSL', 'CABLEMODEM', 'DIALUP', 'FIBRAOPTICA', 'SATELITAL', 'WIRELESS', 'TELEFONIAFIJA', '3G', '4G']:
                if row[field] == '--':
                    row[field] = 'NO'
            
            # Determinar el valor de 'posee_conectividad'
            if all(row[field] == 'NO' for field in ['ADSL', 'CABLEMODEM', 'DIALUP', 'FIBRAOPTICA', 'SATELITAL', 'WIRELESS', 'TELEFONIAFIJA', '3G', '4G']):
                row['posee_conectividad'] = 'NO'
            else:
                row['posee_conectividad'] = 'SI'
            
            writer.writerow(row)

## Ejercicio 3

En el dataset de Lagos (C), se creará una nueva columna llamada **'Sup Tamaño'** que contendrá datos cualitativos indicando el tamaño de cada lago en función de su superficie en kilómetros cuadrados (km²). Esta columna se completará con las palabras: "chico", "medio" o "grande", según los siguientes criterios:

* Lagos con una superficie menor o igual a 17 km² serán clasificados como
"chico".
* Lagos con una superficie mayor que 17 km² y menor o igual a 59 km² serán
clasificados como "medio".
* Lagos con una superficie mayor a 59 km² serán clasificados como "grande".

Además de transformar el campo de coordenadas actual, que sigue el formato estándar de grados, minutos y segundos (GMS), por ejemplo, 42°9'3"S 71°38'59"O, en dos campos separados para la latitud y longitud, añade dos nuevos campos para representar la latitud y longitud en formato de grados decimales (GD). Asegúrate de proporcionar la conversión correcta de GMS a GD para ambos campos. En el ejemplo mencionado el valor resultante debería ser -42.150833 para la latitud y -71.649722 para la longitud.

In [1]:
import csv
from paths import LAGOS_DATA, LAGOS_COPY_DATA
from preprocess_functions import classify_size
from preprocess_functions import convert_gms_to_gd
    
with open(LAGOS_DATA, 'r', newline='', encoding='utf-8') as infile, open(LAGOS_COPY_DATA, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['Sup Tamaño', 'Latitud GD', 'Longitud GD']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:

            # Obtener la superficie del lago del campo 'superficie' como float
            superficie_km2 = float(row['Superficie (km²)'])
            # Categorizar el tamaño del lago
            lake_size = classify_size(superficie_km2)
            # Agregar la nueva columna 'Sup Tamaño' al diccionario de la fila
            row['Sup Tamaño'] = lake_size

            # Convertir coordenadas de GMS a GD para latitud
            latitude_gd = convert_gms_to_gd(row['Coordenadas'].split()[0])
            row['Latitud GD'] = latitude_gd

            # Convertir coordenadas de GMS a GD para longitud
            longitude_gd = convert_gms_to_gd(row['Coordenadas'].split()[1])
            row['Longitud GD'] = longitude_gd
            
            # Escribir la fila en el archivo de salida
            writer.writerow(row)

## Ejercicio 4

Para el dataset de Población del censo 2022 (D) reemplazar los valores "///" y "-" por cero en los campos que corresponda.

Además agregar un nuevo campo que tenga el porcentaje de población en situación de calle. Tener en cuenta el total general (NO tener en cuenta los totales por sexo registrado al nacer).

In [4]:
import csv
from paths import C2022_RESUMEN_DATA, C2022_RESUMEN_COPY_DATA
from preprocess_functions import replace_invalid_values
from preprocess_functions import calculate_porcentaje_situacion_calle

# Abrir el archivo de entrada y crear el archivo de salida
with open(C2022_RESUMEN_DATA, 'r', encoding='utf-8') as infile, open(C2022_RESUMEN_COPY_DATA, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = reader.fieldnames + ['Porcentaje Población en Situación de Calle']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    # Procesar cada fila del archivo de entrada
    for row in reader:
        # Reemplazar los valores inválidos en cada campo
        for field in row:
            row[field] = replace_invalid_values(row[field])
        # Calcular el porcentaje de población en situación de calle
        percentage_homeless = calculate_porcentaje_situacion_calle(row)
        row['Porcentaje Población en Situación de Calle'] = percentage_homeless

        # Escribir la fila modificada en el archivo de salida
        writer.writerow(row)